In [1]:
import os
import csv
import subprocess

#### Functions for saving output files

In [21]:
# Copy Log file
def copy_log_files(cmd_base, exp_folder, exp_name, alg, rep, apdx, prefix, debug):
    if 'vi' in alg:
        return
    source =  prefix+'log_'+alg+'.log'
    target = exp_folder+exp_name+'/'+exp_name+'_log_'+alg+'_'+rep+apdx+'.log'
    command = cmd_base+source+' '+target
    if debug:
        print(command)
    os.system(command)
            
# Copy VI files
def copy_vi_files(cmd_base, exp_folder, exp_name, alg, rep, apdx, prefix, debug):
    if 'vi' in alg:
        return
    source =  prefix+'distr_'+alg+'_'+rep+'.csv'
    target = exp_folder+exp_name+'/'+exp_name+'_distr_'+alg+'_'+rep+apdx+'.csv'
    command = cmd_base+source+' '+target
    if debug:
        print(command)
    os.system(command)

def copy_trace_files(cmd_base, exp_folder, exp_name, alg, rep, apdx, prefix,debug):
    # Copy trace files
    trace_path = prefix+ 'tests/traces/'
    source = trace_path+'distr_'+alg+'_'+rep.upper()+'_trace.csv'
    target = exp_folder+exp_name+'/'+exp_name+'_distr_'+alg+'_'+rep+apdx+'_trace.csv'
    command = cmd_base+source+' '+target
    if debug:
        print(command)
    os.system(command)

def copy_dtmc_files(cmd_base, exp_folder, exp_name, alg, rep, apdx, prefix, debug):
    # Copy DTMC files
    source = prefix+'distr_dtmc_'+alg+'.csv'
    target = exp_folder+exp_name+'/'+exp_name+'_distr_dtmc_'+alg+'_'+rep+apdx+'.csv'
    command = cmd_base+source+' '+target
    if debug:
        print(command)
    os.system(command)

#### Run experiments 
example command:
prism/bin/prism -javastack 80m -javamaxmem 14g prism/tests/corridor.prism prism/tests/corridor.props -distrmethod c51 -prop 3 -v -ex -exportstrat stdout -mainlog prism/log_exp.log

##### Constants

In [15]:
prefix='prism/'
prism_exec = prefix+'bin/prism'
mem_alloc='-javastack 80m -javamaxmem 14g '
experiment_folder = prefix+'tests/experiments/'
rep_base = ' -distrmethod '
tail = ' -v -ex -exportstrat stdout'
log_cmd = ' -mainlog '+prefix+'log_'
atoms_c51 = 101; atoms_qr=1000; def_vmax = 100; big_atoms_c51=501; def_eps=0.00001
atom_vals={'c51':[1, 11, 26, 51, 76, 101, 1001], 'qr': [1, 10, 25, 50, 75, 100, 500, 1000, 5000]}
b_atom_vals =[1, 11, 26, 51, 76, 101]
header_vi =['atoms', 'vmin', 'vmax', 'error', 'epsilon']
header_b = ['atoms', 'bmin', 'bmax']
config = {
    'test': {'model':prefix+'tests/corridor.prism', 'props':prefix+'tests/corridor.props', 'pn':[3,2], 'vmax': def_vmax, 'epsilon':def_eps},
    'cliffs' : {'model':prefix+'tests/cliffs_v2.prism', 'props':prefix+'tests/cliffs_v2.props', 'pn':[3,2], 'vmax': def_vmax, 'epsilon':def_eps},
    'drones' :{'model':prefix+'tests/drones.prism', 'props':prefix+'tests/drones.props', 'pn':[1,2], 'vmax': def_vmax, 'epsilon':def_eps},
    'gridmap10' : {'model':prefix+'tests/gridmap/gridmap_10_10_v2.prism', 'props':prefix+'tests/gridmap/gridmap_10_10.props', 'pn':[3,2], 'vmax': def_vmax, 'epsilon':def_eps},
    'mud_nails' : {'model':prefix+'tests/mud_nails.prism', 'props':prefix+'tests/mud_nails.props', 'pn':[3,2], 'vmax': def_vmax, 'epsilon':def_eps},
    'uav_phi3': {'model':prefix+'tests/uav.prism', 'props':prefix+'tests/uav.props', 'pn':[8,9],  'vmax': 1000, 'epsilon':def_eps},
    'drones_50': {'model':prefix+'tests/drones_50.prism', 'props':prefix+'tests/drones.props', 'pn':[1,2], 'vmax': 800, 'epsilon':0.005},
    'drones_25': {'model':prefix+'tests/drones_30.prism', 'props':prefix+'tests/drones.props', 'pn':[1,2],  'vmax': 800, 'epsilon':0.005},
    'grid_350': {'model':prefix+'tests/gridmap/gridmap_350_2500.prism', 'props':prefix+'tests/gridmap/gridmap_350_2500.props', 'pn':[3,2], 'vmax': 1000, 'epsilon':0.005}
}

##### Case studies to run 

In [16]:
experiment_names=['test'] #['test', 'cliffs', 'drones', 'gridmap10', 'mud_nails']
big_experiments = ['uav_phi3', 'drones_50', 'grid_350']
all_experiments = experiment_names+big_experiments
apdx = '_run'
rep_types = ['c51', 'qr'] # 'c51'
alg_types=['exp'] # ['exp', 'cvar']
cmd_base_copy = "cp "


In [17]:
def create_params(atoms, v_bounds, error, epsilon, b_atoms, b_bounds):
    # make rows
    row_vi = [atoms]+[ v for v in v_bounds]+[error, epsilon] 
    row_b = [b_atoms]+[ b for b in b_bounds]
    print(f'Params: vi:{row_vi}, b:{row_b}')
    
    # saving to params_vi.csv
    with open(prefix+'tests/params_vi.csv', 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)

        write.writerow(header_vi)
        write.writerow(row_vi)
        
    # saving to params_b.csv
    with open(prefix+'tests/params_b.csv', 'w') as f:
        # using csv.writer method from CSV package
        write = csv.writer(f)

        write.writerow(header_b)
        write.writerow(row_b)

#### Base experiment

In [26]:
debug =False
for exp in experiment_names:
    for alg in alg_types:
        for ir, rep in enumerate(rep_types):
            print(f'\nRunning base experiment:{exp}, alg:{alg}, rep:{rep}')
            # create parameters
            if 'c51' in rep:
                create_params(atoms_c51, [0, config[exp]['vmax']], 0.01, config[exp]['epsilon'], atoms_c51, [0, config[exp]['vmax']])
            else:
                create_params(atoms_qr, [0, config[exp]['vmax']], ((1.0/atoms_qr)*10 if 'uav' not in exp else 0.07), config[exp]['epsilon'], atoms_c51, [0, config[exp]['vmax']])
                
            # create cmd + run
            base_command = mem_alloc+config[exp]['model']+' '+config[exp]['props']+rep_base+rep
            options =' -prop '+str(config[exp]['pn'][ir])+tail+log_cmd+alg+'.log'
            if debug:
                print(prism_exec+' '+base_command+options)
            r=os.system(prism_exec+' '+base_command+options)
            print(f'Return code: {r}')

            if r==0:
                # save output for prism vi
                if 'exp' in alg:
                    print("... Saving PRISM VI output files...")
                    copy_trace_files(cmd_base_copy, experiment_folder, exp, 'vi', rep, '', prefix, debug)
                    copy_dtmc_files(cmd_base_copy, experiment_folder, exp, 'vi', rep, '', prefix, debug)

                # save output for current algorithm
                print(f"... Saving {alg} VI output files...")
                copy_log_files(cmd_base_copy, experiment_folder, exp, alg, rep, '', prefix, debug)
                copy_vi_files(cmd_base_copy, experiment_folder, exp, alg, rep, '', prefix, debug)
                copy_trace_files(cmd_base_copy, experiment_folder, exp, alg, rep, '', prefix, debug)
                copy_dtmc_files(cmd_base_copy, experiment_folder, exp, alg, rep, '', prefix, debug)
            else:
                print(f'Error running experiment')
                
                


Running base experiment:test, alg:exp, rep:c51
Params: vi:[101, 0, 100, 0.01, 1e-05], b:[101, 0, 100]
Return code: 0
... Saving PRISM VI output files...
... Saving exp VI output files...

Running base experiment:test, alg:exp, rep:qr
Params: vi:[1000, 0, 100, 0.01, 1e-05], b:[101, 0, 100]
Return code: 0
... Saving PRISM VI output files...
... Saving exp VI output files...
